<a href="https://colab.research.google.com/github/TomoShishido/abltransactivation1/blob/main/Colab_AlphaFold1-2_PAE_json2excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
!pip install openpyxl

In [1]:
import pandas as pd
import os
import glob
import math
import openpyxl as px
from openpyxl.styles import PatternFill
import json

In [2]:
def AlphaFold1to2_PAE_json2excel(AlphaFold1to2_PAE_JSONPATH):

    if os.path.exists(AlphaFold1to2_PAE_JSONPATH):
        print(f'JSON_FILE_PATH: {AlphaFold1to2_PAE_JSONPATH}')

        with open(AlphaFold1to2_PAE_JSONPATH, 'r') as json_file:
            json_content = json_file.read()

        try:
            json_content = json.loads(json_content)
            df_json = pd.json_normalize(json_content) 
        except ValueError as e:
            print(f'Error reading JSON: {e}')
            return 0

        dirname = os.path.dirname(AlphaFold1to2_PAE_JSONPATH)
        namewithoutext = os.path.splitext(os.path.basename(AlphaFold1to2_PAE_JSONPATH))[0]
        json_ext = os.path.splitext(os.path.basename(AlphaFold1to2_PAE_JSONPATH))[1]
        json_filepath = dirname + '/' + namewithoutext + json_ext

        nestedlist1 = None
        if 'predicted_aligned_error' in df_json.columns and df_json['predicted_aligned_error'][0]:
            nestedlist1 = df_json['predicted_aligned_error'][0]
        elif 'pae' in df_json.columns and df_json['pae'][0]:
            nestedlist1 = df_json['pae'][0]
        else:
            print("No suitable key found in JSON")
            return 0

        for i, x in enumerate(nestedlist1):
            for j,y in enumerate(x):
                nestedlist1[i][j] = math.floor(nestedlist1[i][j])

        df1=pd.DataFrame(nestedlist1)
        excel_path =dirname + '/' + namewithoutext + '.xlsx'
        # print(f'excel_path:{excel_path}')
        df1.to_excel(excel_path, sheet_name=namewithoutext, index=False, header=False, startrow=0, startcol=0)
        
        wb = px.load_workbook(filename=excel_path)
        ws = wb[namewithoutext]
        lastrow = ws.max_row
        lastcol = ws.max_column
        for col in range(1, lastcol+1):
            column_letter = px.utils.get_column_letter(col)
            ws.column_dimensions[column_letter].width = 3
            ws.row_dimensions[col].height = 15 #assume in PAE col=row
        for row in range(1, lastrow+1):
            for col in range(1, lastcol+1):
                val = ws.cell(row= row, column= col).value
                very_low = 5
                medium_low =10
                low =15
                slightly_low =20
                if val <= very_low:
                    ws.cell(row=row, column=col).fill = PatternFill(patternType='solid', fgColor='2b381d', bgColor= '2b381d')
                elif val <= medium_low:
                    ws.cell(row=row, column=col).fill = PatternFill(patternType='solid', fgColor='557238', bgColor= '557238')
                elif val <= low:
                    ws.cell(row=row, column=col).fill = PatternFill(patternType='solid', fgColor='80aa55', bgColor= '80aa55')
                elif val <= slightly_low:
                    ws.cell(row=row, column=col).fill = PatternFill(patternType='solid', fgColor='aac78d', bgColor= 'aac78d')
                else:
                    pass
        wb.save(excel_path)
        return excel_path
    else:
        print("There is no JSON_FILE_PATH")
        return 0 

In [ ]:
AlphaFold1to2_PAE_JSONPATH="path_to_your_PAE_json_file"
# AlphaFold1to2_PAE_JSONPATH="PAE/c-crk_c-abl.json"
resultExcelPATH= AlphaFold1to2_PAE_json2excel(AlphaFold1to2_PAE_JSONPATH)
print(f'Excel PATH: {resultExcelPATH}')

JSON_FILE_PATH: PAE/c-crk_c-abl.json
Excel PATH: PAE/c-crk_c-abl.xlsx
